In [ ]:
from google.colab import drive
import os

# 1️⃣ Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import wandb

In [125]:
import pandas as pd

In [2]:
!pip install plotly

In [3]:
import plotly.graph_objects as go

In [4]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: birdyyybai (birdyyybai-university-of-michigan) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
import math
import inspect
from dataclasses import dataclass
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn import functional as F

In [6]:
vocab = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', '+', '&', '*']
device = 'cuda' if torch.cuda.is_available() else 'cpu'
padding_token_index = 13
end_token_index = 12

In [7]:
# create a mapping from chars to ints
stoi = {ch:i for i, ch in enumerate(vocab)}
itos = {i:ch for i, ch in enumerate(vocab)}
encode = lambda s:[stoi[c] for c in s] # encoder: take a string, output a list of ints
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of ints, output a string

print(encode("1+2=3&"))
print(decode(encode("1+2=3&")))

[1, 11, 2, 10, 3, 12]
1+2=3&


In [ ]:
# # train test split
# train_set_1 = np.random.choice(np.arange(10), 8, replace=False)
# train_set_2 = np.random.choice(np.arange(10, 100), 72, replace=False)
# train_set_3 = np.random.choice(np.arange(100, 1000), 720, replace=False)
# test_1 = np.setdiff1d(np.arange(10), train_set_1)
# test_2 = np.setdiff1d(np.arange(10, 100), train_set_2)
# test_3 = np.setdiff1d(np.arange(100, 1000), train_set_3)
# test_12 = np.concatenate([test_1, test_2])
# test = np.concatenate([test_1, test_2, test_3])
# print(np.sort(train_set_1))
# print(np.sort(train_set_2))
# print(np.sort(test_1))
# print(np.sort(test_2))
# print(np.sort(test))

[0 1 2 4 5 6 8 9]
[10 11 12 13 14 15 16 17 18 19 21 22 23 24 25 26 27 28 30 31 33 34 35 36
 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 58 60 63 64 65
 66 68 69 70 71 74 77 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 98 99]
[3 7]
[20 29 32 37 57 59 61 62 67 72 73 75 76 78 79 80 96 97]
[  3   7  20  29  32  37  57  59  61  62  67  72  73  75  76  78  79  80
  96  97 102 107 118 121 125 130 145 146 151 152 154 161 165 169 171 180
 192 193 197 198 199 200 204 207 208 210 218 219 241 254 268 284 294 295
 296 299 303 306 308 317 318 319 327 329 340 341 342 343 348 365 366 373
 377 381 386 396 399 401 407 408 412 417 422 424 427 428 438 443 455 456
 459 468 474 477 496 514 520 526 533 534 536 538 547 553 555 558 559 564
 572 575 582 585 593 594 599 601 608 611 620 622 624 628 635 638 639 647
 650 652 654 655 658 659 660 667 685 690 705 707 715 716 723 724 726 728
 744 749 752 755 759 764 765 766 771 772 776 778 783 785 786 793 803 813
 824 826 838 839 840 841 844 845 862 865 87

In [80]:
def get_batch(phase=None, batch_size=32, block_size=35, mode='train'):

    if mode == 'train':
      # random choose a and b from set
      if phase != "mix":
        a = np.random.randint(10**(phase-1), 10**(phase), batch_size)
        b = np.random.randint(10**(phase-1), 10**(phase), batch_size)
        c = a + b
      elif phase == "mix":
        exp_a = np.random.choice(np.arange(1, 7), size=batch_size, p=[0.05, 0.1, 0.10, 0.15, 0.25, 0.35])
        exp_b = np.random.choice(np.arange(1, 7), size=batch_size, p=[0.05, 0.1, 0.10, 0.15, 0.25, 0.35])
        a = np.random.randint(10**(exp_a-1), 10**(exp_a), size=batch_size)
        b = np.random.randint(10**(exp_b-1), 10**(exp_b), size=batch_size)
        c = a + b
    else:
      if phase != "mix":
        a = np.random.randint(10**(phase-1), 10**(phase), batch_size)
        b = np.random.randint(10**(phase-1), 10**(phase), batch_size)
        c = a + b
      elif phase == "mix":
        exp_a = np.random.choice(np.arange(1, 7), size=batch_size, p=[0.05, 0.1, 0.10, 0.15, 0.25, 0.35])
        exp_b = np.random.choice(np.arange(1, 7), size=batch_size, p=[0.05, 0.1, 0.10, 0.15, 0.25, 0.35])
        a = np.random.randint(10**(exp_a-1), 10**(exp_a), size=batch_size)
        b = np.random.randint(10**(exp_b-1), 10**(exp_b), size=batch_size)
        c = a + b

    x_list, y_list = [], []
    for i, j, k in zip(a, b, c):
        # construct X: "i+j=k&"
        x_str = f"{i}+{j}={k}&"
        # print(x_str)
        x_encoded = encode(x_str)
        x_padded = x_encoded + [padding_token_index] * (block_size - len(x_encoded))
        x_list.append(torch.tensor(x_padded, dtype=torch.int64))

        # construct Y: "k&"
        y_encoded = encode(x_str)[1:]
        y_encoded.append(end_token_index)
        y_padded = y_encoded + [padding_token_index] * (block_size - len(y_encoded))
        y_list.append(torch.tensor(y_padded, dtype=torch.int64))

    x_tensor = torch.stack(x_list).to(device)
    y_tensor = torch.stack(y_list).to(device)
    return x_tensor, y_tensor

In [36]:
get_batch(phase="mix")

(tensor([[ 7,  4,  5,  ..., 13, 13, 13],
         [ 9,  3,  5,  ..., 13, 13, 13],
         [ 1,  4,  8,  ..., 13, 13, 13],
         ...,
         [ 3,  2,  7,  ..., 13, 13, 13],
         [ 8,  1,  4,  ..., 13, 13, 13],
         [ 4,  9,  2,  ..., 13, 13, 13]], device='cuda:0'),
 tensor([[ 4,  5,  9,  ..., 13, 13, 13],
         [ 3,  5,  9,  ..., 13, 13, 13],
         [ 4,  8, 11,  ..., 13, 13, 13],
         ...,
         [ 2,  7,  7,  ..., 13, 13, 13],
         [ 1,  4,  7,  ..., 13, 13, 13],
         [ 9,  2,  0,  ..., 13, 13, 13]], device='cuda:0'))

In [37]:
class LayerNorm(nn.Module):
    """ LayerNorm but with an optional bias. PyTorch doesn't support simply bias=False """

    def __init__(self, ndim, bias=True): # class constructor
        super().__init__()
        # nn.Parameter, pytorch optimize will update the value of this parameter during training
        self.weight = nn.Parameter(torch.ones(ndim)) # trainable parameter
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None # trainable parameter

    def forward(self, input):
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)

class CausalSelfAttention(nn.Module):
    def __init__(self, n_embd, n_head, dropout, block_size, bias=True):
        super().__init__()
        assert n_embd % n_head == 0, "Embedding dimension must be divisible by the number of heads."

        # Store hyperparameters
        self.n_head = n_head
        self.n_embd = n_embd
        self.dropout = dropout
        self.block_size = block_size

        # Key, Query, Value projections
        self.c_attn = nn.Linear(n_embd, 3 * n_embd, bias=bias)
        # Output projection
        self.c_proj = nn.Linear(n_embd, n_embd, bias=bias)
        # Regularization
        self.attn_dropout = nn.Dropout(dropout)
        self.resid_dropout = nn.Dropout(dropout)

        # Check for Flash Attention availability
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
        if not self.flash:
            print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
            # Causal mask for slow attention
            self.register_buffer(
                "bias",
                torch.tril(torch.ones(block_size, block_size)).view(1, 1, block_size, block_size)
            )

    def forward(self, x):
        B, T, C = x.size()  # Batch size, sequence length, embedding dimension

        # Compute Q, K, V
        q, k, v = self.c_attn(x).split(self.n_embd, dim=2)  # Split into Q, K, V (B, T, n_embd)

        # Reshape for multi-head attention
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # (B, n_head, T, head_size)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # (B, n_head, T, head_size)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # (B, n_head, T, head_size)

        # Flash Attention or fallback to manual implementation
        if self.flash:
            y = torch.nn.functional.scaled_dot_product_attention(
                q, k, v,
                attn_mask=None,
                dropout_p=self.dropout if self.training else 0,
                is_causal=True
            )
        else:
            # Manual attention with causal masking
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))  # Scaled dot product
            att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))  # Apply causal mask
            att = F.softmax(att, dim=-1)  # Normalize attention scores
            att = self.attn_dropout(att)
            y = att @ v  # Apply attention weights to values (B, n_head, T, head_size)

        # Reshape back to original format
        y = y.transpose(1, 2).contiguous().view(B, T, C)  # Reassemble heads

        # Output projection and residual dropout
        y = self.resid_dropout(self.c_proj(y))
        return y

class MLP(nn.Module): # FFN

    def __init__(self, n_embd, dropout, bias=True):
        super().__init__()
        self.c_fc    = nn.Linear(n_embd, 4 * n_embd, bias=bias)
        self.gelu    = nn.GELU() # nonlinear activation function
        self.c_proj  = nn.Linear(4 * n_embd, n_embd, bias=bias)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

class Block(nn.Module):
    def __init__(self, n_embd, n_head, dropout, block_size, bias=True):
        super().__init__()
        # LayerNorm and CausalSelfAttention with explicit parameters
        self.ln_1 = LayerNorm(n_embd, bias=bias)
        self.attn = CausalSelfAttention(n_embd, n_head, dropout, block_size, bias=bias)
        self.ln_2 = LayerNorm(n_embd, bias=bias)
        self.mlp = MLP(n_embd, dropout, bias=bias)  # MLP with explicit parameters

    def forward(self, x):
        # Apply residual connection and pre-normalization
        x = x + self.attn(self.ln_1(x))  # Apply LayerNorm before attention
        x = x + self.mlp(self.ln_2(x))  # Apply LayerNorm before MLP
        return x


class GPT(nn.Module):

    def __init__(self, vocab_size, block_size, n_embd, n_layer, n_head, dropout, bias=True):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        super().__init__()
        assert vocab_size is not None
        assert block_size is not None
        self.vocab_size = vocab_size
        self.block_size = block_size
        self.n_embd = n_embd
        self.n_layer = n_layer
        self.n_head = n_head
        self.dropout = dropout
        self.bias = bias

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(vocab_size, n_embd), # token embeddings
            wpe = nn.Embedding(block_size, n_embd), # positional embeddings
            drop = nn.Dropout(dropout),
            h = nn.ModuleList([Block(n_embd, n_head, dropout, block_size, bias=bias) for _ in range(n_layer)]), # a stack of n_layer blocks
            ln_f = LayerNorm(n_embd, bias=bias), # final layer norm
        ))
        self.lm_head = nn.Linear(n_embd, vocab_size, bias=False) # projects the final transformer output to the vocab size

        # init all weights
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.block_size, f"Cannot forward sequence of length {t}, block size is only {self.cblock_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device) # shape (t)

        # forward the GPT model itself
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (t, n_embd)
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        logits = self.lm_head(x)

        loss = None

        if targets is not None:
            # if we are given some desired targets also calculate the loss
            logits = self.lm_head(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=13)
            # inference-time mini-optimization: only forward the lm_head on the very last position
            logits = self.lm_head(x[:, [-1], :]) # note: using list [-1] to preserve the time dim
            # loss = None

        return logits, loss

In [81]:
eval_iters = 200

@torch.no_grad()
def estimate_loss(phase, models):
    out = {}
    models.eval()
    for split in ['train', 'val']:
      losses = torch.zeros(eval_iters)
      for k in range(eval_iters):
          X, Y = get_batch(phase, mode=split)
          padding_mask_x = (X != padding_token_index).long()
          logits, loss = models(X, Y)
          losses[k] = loss.item()
      out[split] = losses.mean()
    models.train()
    return out

In [46]:
# batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 35 # what is the maximum context length for predictions?
max_iters = 50000
# num_epochs = 100
eval_interval = 100
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 20
n_embd = 256
n_head = 4
n_layer = 8
dropout = 0.0
# # torch.manual_seed(1337)
# if torch.cuda.is_available():
#     torch.cuda.manual_seed_all(1337)
bias = True # if using bias inside all Linear layers
vocab_size = len(vocab)

In [40]:
wandb.init(project="transformer", config={
    "learning_rate": 1e-5,
    "batch_size": 32,
    "block_size": 35,
    "optimizer": "AdamW",
    "n_embd": 256,
    "n_head": 4,
    "n_layer": 8,
    "dropout": 0.0,
})

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [47]:
def accuracy(model):
    correct = 0
    for j in range(100):

        a = np.random.choice(np.arange(1000000), 1)
        b = np.random.choice(np.arange(1000000), 1)

        c = a + b
        input = f"{a.item()}+{b.item()}="
        context = torch.tensor(encode(input), dtype=torch.long, device=device)
        output = generate(model, context, 100, 1)
        if output == f"{a.item()}+{b.item()}={c.item()}":
            correct += 1
    print(f"Accuracy for addition: {correct / 100} ")
    return correct / 100

In [48]:
@torch.no_grad()
def generate(model, idx, max_new_tokens, temperature=1.0, top_k=None):
    """
    Generate a sequence of tokens given an initial sequence.

    Parameters:
        model (nn.Module): The model used for generation.
        idx (torch.Tensor or list): Initial sequence of indices (LongTensor of shape (b,t)).
        max_new_tokens (int): Number of new tokens to generate.
        temperature (float): Scaling factor for logits before softmax.
        top_k (int, optional): If specified, restricts sampling to top k tokens.

    Returns:
        torch.Tensor: The generated sequence.
    """
    idx = idx.unsqueeze(0) if idx.dim() == 1 else idx
    idx = torch.tensor(idx, device=model.device) if not isinstance(idx, torch.Tensor) else idx.to(model.device)

    for _ in range(max_new_tokens):
        # Ensure context length does not exceed model's block size
        idx_cond = idx if idx.size(1) <= model.block_size else idx[:, -model.block_size:]

        # Forward pass to get logits
        logits, _ = model(idx_cond)

        # Extract logits for the last token and apply temperature scaling
        logits = logits[:, -1, :] / temperature

        # Apply top-k filtering if necessary
        if top_k is not None:
            v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
            logits[logits < v[:, [-1]]] = -float('Inf')

        # Convert logits to probabilities
        probs = F.softmax(logits, dim=-1)

        # Sample next token
        idx_next = torch.multinomial(probs, num_samples=1)

        if idx_next == end_token_index:
            break
        # Append sampled token to sequence

        # Append sampled token to sequence
        idx = torch.cat((idx, idx_next), dim=1)

    return decode(idx.tolist()[0])


In [49]:
model = GPT(vocab_size, block_size, n_embd, n_layer, n_head, dropout, bias=bias)
m = model.to(device)

In [50]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

In [82]:
wandb.init(project="transformer", config={
    "learning_rate": 1e-5,
    "batch_size": 32,
    "block_size": 35,
    "optimizer": "AdamW",
    "n_embd": 256,
    "n_head": 4,
    "n_layer": 8,
    "dropout": 0.0
})

In [83]:
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

phase = "mix"
best_acc = 0
counter = 0
best_loss = float('inf')
val_loss_list = []
acc_list = []

patience = 50

for iter in tqdm(range(max_iters), desc="Training Progress"):
    # if iter > 1000:
    #   phase = 2
    # if iter > 3000:
    #   phase = 3
    # if iter > 5000:
    #   phase = 4
    # if iter > 7000:
    #   phase = 5
    # if iter > 9000:
    #   phase = 6
    # if iter > 11000:
    #   phase = "mix"

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(phase, model)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        log_dict = {"Loss": losses['val']}
        val_loss_list.append(round(losses['val'].item(), 4))

        if phase == "mix":
            # acc = accuracy(model)

            # acc_list.append(acc)
            # log_dict["Accuracy"] = acc

            if losses['val'] < best_loss:
                counter = 0
                # best_acc = max(best_acc, acc)
                best_loss = min(best_loss, losses['val'])
            else:
                counter += 1
                if counter >= patience:
                    print(f"Early Stopping at iteration {iter}")
                    break

        # record to W&B
        wandb.log(log_dict)

    # sample a batch of data

    xb, yb = get_batch(phase)

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


6.33472 M parameters


Training Progress:   0%|          | 7/50000 [00:02<4:05:00,  3.40it/s] 

step 0: train loss 1.2472, val loss 1.2475


Training Progress:   0%|          | 109/50000 [00:07<1:30:58,  9.14it/s]

step 100: train loss 1.2454, val loss 1.2453


...

step 33300: train loss 1.2382, val loss 1.2383


Training Progress:  67%|██████▋   | 33409/50000 [24:26<29:30,  9.37it/s]

step 33400: train loss 1.2388, val loss 1.2383


Training Progress:  67%|██████▋   | 33500/50000 [24:30<12:04, 22.78it/s]

step 33500: train loss 1.2378, val loss 1.2396
Early Stopping at iteration 33500


In [84]:
wandb.finish()

Loss,█▆▇▇▆█▇▆▆▅▆▆▄▅▄▄▆▅▇▃▅▅▅▄▆▃▃▃▄▂▄▁▄▃▁▁▃▂▂▂
Loss,1.23828


In [ ]:
fig = go.Figure()

iterations = list(range(58))

# Add lines
fig.add_trace(go.Scatter(x=iterations, y=val_loss_list, mode='lines+markers', name='Validation Loss'))
fig.add_trace(go.Scatter(x=iterations, y=acc_test_list, mode='lines+markers', name='Test Accuracy'))
fig.add_trace(go.Scatter(x=iterations, y=acc_all_list, mode='lines+markers', name='All Accuracy'))

# Set title label
fig.update_layout(
    title="Validation Loss & Accuracy over Iterations",
    xaxis_title="Iterations",
    yaxis_title="Metrics",
    legend_title="Metrics",
    width = 900,
    height = 500
)

fig.show()

wandb.init(project="transformer", name="val_loss and accuracy plot")
wandb.log({"Interactive Chart": wandb.Html(fig.to_html())})
wandb.finish()

In [129]:
def accuracy_print(model, num_digits, need_print=False):
        correct = 0

        for j in range(100):
            a = np.random.choice(np.arange(10**num_digits), 1)
            b = np.random.choice(np.arange(10**num_digits), 1)
            c = a + b
            input = f"{a.item()}+{b.item()}="
            context = torch.tensor(encode(input), dtype=torch.long, device=device)
            output = generate(model, context, 100, 1)
            if need_print and j // 10 == 0:
                print(f"Input: {input}")
                print(f"Output: {output}")
            if output == f"{a.item()}+{b.item()}={c.item()}":
                correct += 1
        acc = correct / 100
        print(f"Accuracy for {num_digits} digits addition: {acc} ")
        return acc

In [130]:
accuracy_print(model, 1)

Accuracy for 1 digits addition: 0.97 


0.97

In [131]:
accuracy_print(model, 2)

Accuracy for 2 digits addition: 1.0 


1.0

In [132]:
accuracy_print(model, 3)

Accuracy for 3 digits addition: 0.94 


0.94

In [133]:
accuracy_print(model, 4)

Accuracy for 4 digits addition: 0.92 


0.92

In [134]:
accuracy_print(model, 5)

Accuracy for 5 digits addition: 0.8 


0.8

In [135]:
accuracy_print(model, 6)

Accuracy for 6 digits addition: 0.91 


0.91

In [136]:
accuracy_print(model, 7)

Accuracy for 7 digits addition: 0.02 


0.02

In [137]:
accuracy_print(model, 8)

Accuracy for 8 digits addition: 0.0 


0.0

In [138]:
def get_avg_performance(model):
    dict_acc = {}
    for num_dig in range(1, 9):
        dict_acc[num_dig] = accuracy_print(model, num_dig, need_print=False)
    return dict_acc

In [139]:
avg_performance = get_avg_performance(model)

Accuracy for 1 digits addition: 0.98 
Accuracy for 2 digits addition: 0.99 
Accuracy for 3 digits addition: 0.97 
Accuracy for 4 digits addition: 0.92 
Accuracy for 5 digits addition: 0.8 
Accuracy for 6 digits addition: 0.91 
Accuracy for 7 digits addition: 0.03 
Accuracy for 8 digits addition: 0.0 


In [142]:
x_values = list(avg_performance.keys())
y_values = list(avg_performance.values())


fig = go.Figure(go.Bar(x=x_values, y=y_values, marker_color='lightskyblue'))


fig.update_layout(
    title="Accuracy for different digits addition",
    xaxis_title="Num Digits",
    yaxis_title="Accuracy",
    template="plotly_white",
    width=800,
    height= 500
)


fig.show()

wandb.init(project="transformer", name="Accuracy for different digits addition plot")
wandb.log({"Interactive Chart": wandb.Html(fig.to_html())})
wandb.finish()

In [ ]:
import subprocess

os.system('git config --global user.email "zifeibai@umich.edu"')
os.system('git config --global user.name "ZifeiBai"')

# 2️⃣ **Use Google Drive to store GitHub Token**
GITHUB_TOKEN_PATH = "/content/drive/MyDrive/URPS/github_token.txt"
if os.path.exists(GITHUB_TOKEN_PATH):
    with open(GITHUB_TOKEN_PATH, "r") as f:
        os.environ["GITHUB_TOKEN"] = f.read().strip()
else:
    print("❌ GitHub Token")
    exit(1)

# 3️⃣ **Set up GitHub remote repo**
GIT_PATH = "/content/drive/MyDrive/URPS/Git"
REPO_URL = f"https://{os.environ['GITHUB_TOKEN']}@github.com/ZifeiBai/URPS.git"

if not os.path.exists(GIT_PATH):
    print(f"📁 Creating directory: {GIT_PATH}")
    os.makedirs(GIT_PATH)

# 4️⃣ **If .git/ does not exsit， need to clone**
if not os.path.exists(os.path.join(GIT_PATH, ".git")):
    print("❌ Git repository not found. Cloning...")
    subprocess.run(f"rm -rf {GIT_PATH}", shell=True, check=True)
    subprocess.run(f"git clone {REPO_URL} {GIT_PATH}", shell=True, check=True)

# 5️⃣ **Enter Git repo**
os.chdir(GIT_PATH)
print("📂 Changed working directory to:", os.getcwd())


# 6️⃣ **Check Git status**
status_output = subprocess.run("git status", shell=True, capture_output=True, text=True)
print(status_output.stdout)

#  **Push to Git**
print("🚀 Adding files to Git...")
subprocess.run("git add .", shell=True, check=True)

print("📝 Committing changes...")
commit_output = subprocess.run('git commit -m "Auto update from Google Colab 2.6"', shell=True, capture_output=True, text=True)
print(commit_output.stdout)



print("📤 Pushing to GitHub...")
push_output = subprocess.run("git push origin main", shell=True, capture_output=True, text=True)
if "fatal" in push_output.stderr or "error:" in push_output.stderr:
    print("❌ Real Git Push Error:", push_output.stderr)
else:
    print("✅ Git Push Success!")

📂 Changed working directory to: /content/drive/MyDrive/URPS/Git
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   transformer.ipynb

no changes added to commit (use "git add" and/or "git commit -a")

🚀 Adding files to Git...
📝 Committing changes...
[main 1fa1a0f] Auto update from Google Colab 2.6
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite transformer.ipynb (94%)

📤 Pushing to GitHub...
✅ Git Push Success!
